## Step 3: run for PypeIt extraction.

###  prior to running this notebook:

- Run through Step_2_combine_msaexp_and_pypeit.ipynb
- Install PypeIt
- Download the PypeIt-development-suite: https://github.com/pypeit/PypeIt-development-suite
- Don't forget to put utils_myue.py into this directory.

In [4]:
import os, sys

# note: the PypeIt-development-suite is constantly changing. 
# If your version doesn't work, please contact me at yuemh@arizona.edu, I will share my version (which works) with you.
sys.path.append('/Users/minghao/Research/Projects/JWST/dependencies/PypeIt-development-suite/dev_algorithms/jwst/')

from pathlib import Path
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy.table import Table

import glob

import utils_myue
import importlib
importlib.reload(utils_myue)

<module 'utils_myue' from '/Users/minghao/Research/Projects/JWST/ID4713_add/reduction_hack_code/utils_myue.py'>

In [5]:
import pypeit
pypeit.__version__

'1.17.1'

In [13]:
def reduce_pointing(prefix, suffix, spec2dir, outdir=None, fix_persistence=True,\
                    bad_bkg_sources = {}, coadd2d_kwargs = {},\
                    ibkg_list_default=[[1,2], [0,2], [0,1], [4,5], [3,5], [3,4]],\
                   coadd2d_command='pypeit_coadd_2dspec', objids=None, **kwargs):

    if outdir is None:
        outdir = prefix

    outdir = os.path.abspath(outdir)
    
    scipath = os.path.join(outdir, 'Science')
    if not os.path.isdir(scipath):
        #msgs.info('Creating directory for Science output: {0}'.format(scipath))
        os.makedirs(scipath)

    coaddpath = outdir + '/Science_coadd/'
    if not os.path.isdir(coaddpath):
        os.system('mkdir -p %s'%coaddpath)

    nscleanfiles_1 = glob.glob(f'{spec2dir}/{prefix}*nrs1*{suffix}')
    scifiles_1 = [f.replace('_nsclean', '') for f in nscleanfiles_1]
    scifiles_1.sort()
    print(scifiles_1)

    msareducer = utils_myue.MSAreducer(scifiles_1, outdir)#, from_msaexp=True)
    
    for ii, (islit, isource) in enumerate(msareducer.slits_sources):

        if isource in bad_bkg_sources.keys():
            ibkg_list = bad_bkg_sources[isource]['bkg_indice']
        else:
            ibkg_list = ibkg_list_default

        if objids is not None:
            if not isource in objids:
                continue

        msareducer.reduce_slit(islit, isource, ibkg_list=ibkg_list)
        
        # step 2: coadd slits
        if isource in coadd2d_kwargs.keys():
            msareducer.coadd2d_source(islit, isource, outdir=coaddpath,\
                coadd2d_command=coadd2d_command,
                **coadd2d_kwargs[isource], **kwargs)
        else:
            msareducer.coadd2d_source(islit, isource, outdir=coaddpath,\
                coadd2d_command=coadd2d_command,\
                find_trim_edge='3,3', find_min_max='400,800', **kwargs)
            
        # construct manual extraction for each object
        msareducer.manual_extraction(islit, isource)
    
    return msareducer

In [36]:
# something is wrong with coadd2d
# test it
outdir = './msaexp_pypeit/jw04713002001_msaexp/'

for ii, (islit, isource) in enumerate(reducer_pointing_1.slits_sources):
    print(islit, isource)
    reducer_pointing_1.coadd2d_source(islit, isource, outdir=outdir + '/Science_coadd/',\
                coadd2d_command='pypeit_coadd_2dspec',\
                find_trim_edge='3,3', find_min_max='400,800', offsets=[0, 0.529])
    break

10 4713_11362


[INFO]    :: Loading the reduction file
[INFO]    :: Coadd2D input file loaded successfully.
[INFO]    :: .coadd2d file successfully vetted.
[INFO]    :: Reducing target jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_11362
             Coadding frame sky-subtracted with DIFF
             Searching for objects that are POS_NEG
             Combining frames in 2d coadd:
             Exp 0: spec2d_jw04713002001_03101_00002_source_4713_11362.fits
             Exp 1: spec2d_jw04713002001_03101_00003_source_4713_11362.fits
             
[INFO]    :: Detectors to work on: [1]
[INFO]    :: Working on detector 1
[INFO]    :: iref for the list is set to 0
[INFO]    :: The shape of the list is: (6, 769)
[INFO]    :: shape of the first wave_grid in the list is: (769,)
[INFO]    :: Get Weights
[INFO]    :: Using uniform weights
[INFO]    :: Get Offsets
[INFO]    :: Using user input offsets
[INFO]    :: 
             -----------------------------------------------

Writing full parameter set to /Users/minghao/Research/Projects/JWST/ID4713_add/reduction/jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_11362_coadd2d.par.
copying: 
./msaexp_pypeit/jw04713002001_msaexp//Science_coadd//spec2d_jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_11362.fits


cp: Science_coadd/spec1d_jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_11362.fits: No such file or directory


In [ ]:
# J0100 field has two objects that requires special treatment

bad_bkg_sources_1 = {}

coadd2d_kwargs_1 = {}

bad_bkg_sources_2 = {'4713_11981': {'bkg_indice':[[2],[0,2],[1], [5],[3,5],[4]]},
                   '4713_11743': {'bkg_indice':[[1],[2],[0], [4],[5],[3]]},}

# this dict stores objects that need specific coadd2d and auto extaction treatment
coadd2d_kwargs_2 = {'4713_6733':{'offsets':[0,0], 'stack_indices':[0,3],\
                               'find_trim_edge':'5,5', 'find_min_max':'400,800'},
                 '4713_11743':{'offsets':[0,-5.28973998,0,-5.28973998], 'stack_indices':[0,1,3,4]}}

In [35]:
reducer_pointing_1 = reduce_pointing(prefix='jw04713002001', suffix='_nsclean.fits',\
                                    spec2dir='./spec2pipe/',\
                                    outdir='./msaexp_pypeit/jw04713002001_msaexp/',
                                    objids=None,\
                                    ibkg_list_default=[[1], [0]], 
                                    offsets=[0, 0.529])

['./spec2pipe/jw04713002001_03101_00002_nrs1.fits', './spec2pipe/jw04713002001_03101_00003_nrs1.fits']


KeyboardInterrupt: 

In [ ]:
reducer_pointing_1.find_islit_by_isource('4713_16184')

### after getting the two pointing reducers, we can save the info to a master table

In [17]:

reducer_pointing_1.saveinfo(tbl_info)
reducer_pointing_1.tbl_extraction_info['pointing'] = 1
#reducer_pointing_2.saveinfo(tbl_info)

In [19]:
reducer_pointing_1.tbl_extraction_info.write('./msaexp_pypeit/masterinfo_J0100_add_pypeit_msaexp.fits', overwrite=True)

In [ ]:
masterinfo.write('./data-release-2/masterinfo_J0100_pypeit_msaexp.fits', overwrite=True)

### rename and coadd 1d specs

Since we have two visits, and the two visits have overlapped objects, we coadd the spectra to improve the S/N.

In [20]:
masterinfo = Table.read('./msaexp_pypeit/masterinfo_J0100_add_pypeit_msaexp.fits')

In [22]:
def stack_1dspec(filelist, outfile):

    script_string =\
f'''
[coadd1d]
  coaddfile='{outfile}'
  ex_value = 'OPT'
  flux_value = False

coadd1d read
  filename | obj_id
'''
    for fname in filelist:
        with fits.open(fname) as hdulist:
            header = hdulist[1].header
            objid = header['NAME']
            script_string += f'{fname} | {objid}\n'

    script_string +=\
'''
coadd1d end
'''
    with open('coadd1dscript.txt', 'w') as script:
        script.write(script_string)
    os.system('pypeit_coadd_1dspec coadd1dscript.txt')

In [23]:
!mkdir ./msaexp_pypeit/reduced_spec_msaexp_pypeit

In [27]:
from collections import Counter

isources_counter = Counter(masterinfo['isource'])
savedir = './msaexp_pypeit/reduced_spec_msaexp_pypeit/'

for isource, count in isources_counter.items():
    #if not isource=='4713_15612':
    #    continue
    #else:
    #    print(isource)
    if count == 1:
        #continue
        subtbl = masterinfo[masterinfo['isource']==isource]
        assert len(subtbl)==1

        pointing = subtbl['pointing'][0]
        #datadir = f'./msaexp_pypeit/jw0471300100{pointing}_msaexp/Science_coadd'
        spec2dfile = subtbl['spec2dlist'][0]
        spec1dfile = subtbl['spec1dlist'][0]
        manu1dfile = subtbl['manual1dlist'][0]

        spec1dlist = [spec1dfile]
        manu1dlist = [manu1dfile]

        os.system(f'cp {spec2dfile} {savedir}spec2d_{isource}_pointing{pointing}.fits')
        os.system(f'cp {spec1dfile} {savedir}spec1d_{isource}_pointing{pointing}.fits')
        os.system(f'cp {manu1dfile} {savedir}manu1d_{isource}_pointing{pointing}.fits')

        if os.path.exists(spec1dfile):
            stack_1dspec(spec1dlist, f'spec1d_{isource}_stack.fits')
            os.system(f'mv spec1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')

        
        stack_1dspec(manu1dlist, f'manu1d_{isource}_stack.fits')
        os.system(f'mv manu1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')
    else:
        print(isource)
        #continue
        subtbl = masterinfo[masterinfo['isource']==isource]
        # perform pypeit stacking
        spec1dlist = []
        manu1dlist = []
        for idx in range(len(subtbl)):
            pointing = subtbl['pointing'][idx]
            direct = f'./data-release-2/jw0471300100{pointing}_msaexp/Science_coadd/'
            
            spec2dfile = os.path.abspath(os.path.join(direct, subtbl['spec2dlist'][idx]))
            spec1dfile = os.path.abspath(os.path.join(direct, subtbl['spec1dlist'][idx]))
            spec1dlist.append(spec1dfile)
            manu1dlist.append(spec1dfile.replace('spec1d', 'manual1d'))

            os.system(f'cp {spec2dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec2d_{isource}_pointing{pointing}.fits')
            os.system(f'cp {spec1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/spec1d_{isource}_pointing{pointing}.fits')
            os.system(f'cp {manu1dfile} ./data-release-2/reduced_spec_msaexp_pypeit/manu1d_{isource}_pointing{pointing}.fits')

        if os.path.exists(spec1dlist[0]) and os.path.exists(spec1dlist[1]):
            stack_1dspec(spec1dlist, f'spec1d_{isource}_stack.fits')
            os.system(f'mv spec1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')
        else:
            
            stack_1dspec(manu1dlist, f'manu1d_{isource}_stack.fits')
            os.system(f'mv manu1d_{isource}_stack.fits ./data-release-2/reduced_spec_msaexp_pypeit/')

        # copy spec2d 
        

cp: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction/msaexp_pypeit/jw04713002001_msaexp/Science_coadd//spec1d_jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_11362.fits: No such file or directory
[INFO]    :: Loading the reduction file
[INFO]    :: Coadd1D input file loaded successfully.
[INFO]    :: Using a sn_smooth_npix=77 to decide how to scale and weight your spectra
[INFO]    :: Computing weights with weight_method=auto
[INFO]    :: Using constant weights for coadding, S/N = 0.01, weight = 0.01 for 0th exposure
[WARNING] :: Error renormalization found correction factor sigma_corr = 0.087536 < 1.
              Errors are overestimated so not applying correction
[INFO]    :: File written to: manu1d_4713_11362_stack.fits
[INFO]    :: Coadding complete


Writing the parameters to coadd1d.par


mv: rename manu1d_4713_11362_stack.fits to ./data-release-2/reduced_spec_msaexp_pypeit/: No such file or directory
cp: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction/msaexp_pypeit/jw04713002001_msaexp/Science_coadd//spec1d_jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_12133.fits: No such file or directory
[INFO]    :: Loading the reduction file
[INFO]    :: Coadd1D input file loaded successfully.
[WARNING] :: Selected configuration file already exists and will be overwritten!
[INFO]    :: Using a sn_smooth_npix=84 to decide how to scale and weight your spectra
[INFO]    :: Computing weights with weight_method=auto
[INFO]    :: Using constant weights for coadding, S/N = 0.24, weight = 0.06 for 0th exposure
[WARNING] :: Error renormalization found correction factor sigma_corr = 0.120789 < 1.
              Errors are overestimated so not applying correction
[INFO]    :: File written to: manu1d_4713_12133_stack.fits
[INFO]    :: Coadding com

Writing the parameters to coadd1d.par


mv: rename manu1d_4713_12133_stack.fits to ./data-release-2/reduced_spec_msaexp_pypeit/: No such file or directory
[INFO]    :: Loading the reduction file
[INFO]    :: Coadd1D input file loaded successfully.
[WARNING] :: Selected configuration file already exists and will be overwritten!
[INFO]    :: Using a sn_smooth_npix=87 to decide how to scale and weight your spectra
[INFO]    :: Computing weights with weight_method=auto
[INFO]    :: Using constant weights for coadding, S/N = 0.47, weight = 0.22 for 0th exposure
[WARNING] :: Error renormalization found correction factor sigma_corr = 0.176569 < 1.
              Errors are overestimated so not applying correction
[INFO]    :: File written to: spec1d_4713_10348_stack.fits
[INFO]    :: Coadding complete


...

mv: rename spec1d_4713_10086_stack.fits to ./data-release-2/reduced_spec_msaexp_pypeit/: No such file or directory
[INFO]    :: Loading the reduction file
[INFO]    :: Coadd1D input file loaded successfully.
[WARNING] :: Selected configuration file already exists and will be overwritten!
[INFO]    :: Using a sn_smooth_npix=77 to decide how to scale and weight your spectra
[INFO]    :: Computing weights with weight_method=auto
[INFO]    :: Using constant weights for coadding, S/N = 0.39, weight = 0.15 for 0th exposure
[WARNING] :: Error renormalization found correction factor sigma_corr = 0.082251 < 1.
              Errors are overestimated so not applying correction
[INFO]    :: File written to: manu1d_4713_10086_stack.fits
[INFO]    :: Coadding complete


Writing the parameters to coadd1d.par


mv: rename manu1d_4713_10086_stack.fits to ./data-release-2/reduced_spec_msaexp_pypeit/: No such file or directory
cp: /Users/minghao/Research/Projects/JWST/ID4713_add/reduction/msaexp_pypeit/jw04713002001_msaexp/Science_coadd//spec1d_jw04713002001_03101_00002_nrs1_nsclean-jw04713002001_03101_00003_nrs1_nsclean-4713_9172.fits: No such file or directory
[INFO]    :: Loading the reduction file
[INFO]    :: Coadd1D input file loaded successfully.
[WARNING] :: Selected configuration file already exists and will be overwritten!
[INFO]    :: Using a sn_smooth_npix=77 to decide how to scale and weight your spectra
[INFO]    :: Computing weights with weight_method=auto
[INFO]    :: Using constant weights for coadding, S/N = 0.01, weight = 0.01 for 0th exposure
[WARNING] :: Error renormalization found correction factor sigma_corr = 0.090173 < 1.
              Errors are overestimated so not applying correction
[INFO]    :: File written to: manu1d_4713_9172_stack.fits
[INFO]    :: Coadding compl

Writing the parameters to coadd1d.par


mv: rename manu1d_4713_9172_stack.fits to ./data-release-2/reduced_spec_msaexp_pypeit/: No such file or directory


At this time, you should have stacked 1d spec (spec1d_{isource}\_stack.fits), force-extracted 1d spec "manu1d_{isource}_stack.fits", and 2d spec for each object in each visit.